In [ ]:
#| default_exp catalog

In [ ]:
#| export
from loguru import logger
from tabulate import tabulate


In [ ]:
#| export
class GPUCatalog():
    def __init__(self, catalog_dict:dict, region:str, subsidiary:str='FR'):
        """
        OVH Project class, storing project information
    
        Parameters
        ----------
        catalog_dict : dict, required
            OVH Catalog dictionary
        region : str, required
            OVH Catalog region
        subsidiary : str, required
            OVH Catalog subsidiary/country, default set to FR (France)
        consumption_mode: str, required
            OVH Ressource consumption mode, default set to consumption. Possible values ['consumption', 'monthly.postpaid']
        """
        self.region = region
        self.subsidiary = subsidiary
        self.catalog_dict = catalog_dict
        self.consumption_mode = None
        self.gpu_families =  ['a10', 'rtx5000', 't1', 't2', 'l4', 'h100']
        self.ressource_type = 'instance'
        self._project_plans = []
        self._instances_addons = []
        self._gpu_addons = []
        self.gpu_instances_data = []

    def _list_project_plans(self):
        """
        Lists OVH Catalog Plans
        """ 
        self._project_plans = [p for p in self.catalog_dict['plans'] if (p['planCode']=='project' and p['invoiceName']=='Public Cloud Project')]

    def _list_addons(self):
        """
        Lists OVH Plans addons
        """ 
        self._instances_addons = [p for p in self._project_plans[0]['addonFamilies'] if p['name']==self.ressource_type]

    def _list_gpu_addons(self, consumption_mode:str='consumption'):
        """
        Lists OVH GPU Addons
        """ 
        dict_consumption_mode = {
            'consumption': 'consumption',
            'monthly.postpaid': 'monthly'
        }
        self._gpu_addons = [i for i in self._instances_addons[0]['addons'] 
                           if (i.split('.')[0].split('-')[0] in self.gpu_families) 
                           and (dict_consumption_mode.get(consumption_mode) in i.split('.')[1])]

    def list_gpu_instances(self, consumption_mode:str='consumption'):
        """
        Lists available gpu instances for the OVH Catalog. It displays a table containing:
        - Name
        - Price (hourly or monthly depending on consumption mode)
        - GPU Model
        - GPU Memory
        - CPU Cores
        - RAM

        Parameters
        ----------
        consumption_mode : str, required, default to consumption (consumption or monthly.postpaid) 
            GPU instance consumption mode
        """
        self._list_project_plans()
        self._list_addons()
        self._list_gpu_addons(consumption_mode)

        dict_consumption_mode = {
            'consumption': 'per hour',
            'monthly.postpaid': 'per month'
        }
        
        self.gpu_instances_data = []
        for addon_code in sorted(self._gpu_addons):
            addon = [p for p in self.catalog_dict['addons'] if p['planCode'] == addon_code]
            if addon:
                a = addon[0]
                self.gpu_instances_data.append({
                    'Name': a['invoiceName'],
                    f'Price {dict_consumption_mode.get(consumption_mode)}': a['pricings'][0]['formattedPrice'],
                    'GPU Model': a['blobs']['technical']['gpu']['model'],
                    'GPU Memory': f"{a['blobs']['technical']['gpu']['memory']['size']}GB",
                    'CPU Cores': a['blobs']['technical']['cpu']['cores'],
                    'RAM': f"{a['blobs']['technical']['memory']['size']}GB"
                })
    
        logger.info("Available GPU instances:")
        print(tabulate(self.gpu_instances_data, headers='keys', tablefmt='grid', showindex=range(1, len(self.gpu_instances_data)+1)))    
        

First let's create an OVHClient to be able to then look into an GPUCatalog

In [ ]:
#| eval: false
from ovhmanager.core import OVHGPUManager
client = OVHGPUManager()

In [ ]:
#| eval: false
client.select_region('BHS')

2026-02-11 11:06:01.659 | INFO     | ovhmanager.manager:select_region:119 - Selected region: BHS


'BHS'

We retrieve an OVH catalog dictionary first

In [ ]:
#| eval: false
catalog_dict = client.retrieve_catalog_dict()

2026-02-11 11:06:03.132 | INFO     | ovhmanager.manager:retrieve_catalog_dict:156 - Retrieved catalog dictionary for FR subsidiary


Then we create the OVH catalog

In [ ]:
#| eval: false
catalog = GPUCatalog(catalog_dict, client.region)
catalog.subsidiary, catalog.region

('FR', 'BHS')

Now let's list the OVH Plans available for Cloud Projects

In [ ]:
#| eval: false
catalog._list_project_plans()
assert len(catalog._project_plans)>0

Next within the plans the available Addons for a ressource of type `instance`

In [ ]:
#| eval: false
catalog._list_addons()
assert len(catalog._instances_addons)>0

Then the specific GPU addons for a specific consumption mode

In [ ]:
#| eval: false
catalog._list_gpu_addons('monthly.postpaid')
assert len(catalog._gpu_addons)>0

Once we have the GPU instance addons for a monthly consumption mode we can list the different GPU types, their prices and other attributes

In [ ]:
#| eval: false
catalog.list_gpu_instances('monthly.postpaid')

2026-02-11 11:06:06.036 | INFO     | __main__:list_gpu_instances:91 - Available GPU instances:


+----+-----------+-------------------+-------------+--------------+-------------+--------+
|    | Name      | Price per month   | GPU Model   | GPU Memory   |   CPU Cores | RAM    |
+====+===========+===================+=============+==============+=============+========+
|  1 | h100-1520 | 7770.00 €         | H100        | 80GB         |         120 | 1520GB |
+----+-----------+-------------------+-------------+--------------+-------------+--------+
|  2 | h100-380  | 1940.00 €         | H100        | 80GB         |          30 | 380GB  |
+----+-----------+-------------------+-------------+--------------+-------------+--------+
|  3 | h100-760  | 3880.00 €         | H100        | 80GB         |          60 | 760GB  |
+----+-----------+-------------------+-------------+--------------+-------------+--------+
|  4 | l4-180    | 1080.00 €         | L4          | 24GB         |          45 | 180GB  |
+----+-----------+-------------------+-------------+--------------+-------------+--------+

Now let's check hourly consumption mode

In [ ]:
#| eval: false
catalog._list_gpu_addons('consumption')
assert len(catalog._gpu_addons)>0

In [ ]:
#| eval: false
catalog.list_gpu_instances('consumption')

2026-02-11 11:06:08.599 | INFO     | __main__:list_gpu_instances:91 - Available GPU instances:


+----+------------+------------------+----------------+--------------+-------------+--------+
|    | Name       | Price per hour   | GPU Model      | GPU Memory   |   CPU Cores | RAM    |
+====+============+==================+================+==============+=============+========+
|  1 | a10-180    | 3.04 €           | A10            | 24GB         |         120 | 180GB  |
+----+------------+------------------+----------------+--------------+-------------+--------+
|  2 | a10-45     | 0.76 €           | A10            | 24GB         |          30 | 45GB   |
+----+------------+------------------+----------------+--------------+-------------+--------+
|  3 | a10-90     | 1.52 €           | A10            | 24GB         |          60 | 90GB   |
+----+------------+------------------+----------------+--------------+-------------+--------+
|  4 | h100-1520  | 11.20 €          | H100           | 80GB         |         120 | 1520GB |
+----+------------+------------------+----------------+-----